# training pipeline

In [ ]:
pip install meshy3d


ERROR: Could not find a version that satisfies the requirement meshy3d (from versions: none)
ERROR: No matching distribution found for meshy3d


In [ ]:
import requests
import json

def generate_3d_model_from_prompt(prompt, api_key, art_style="realistic", should_remesh=True, mode="preview"):
    """
    Generate a 3D model based on a textual prompt using the Meshy API.

    :param prompt: A textual description of the 3D model to generate
    :param api_key: API key for authentication
    :param art_style: Style of the generated 3D model (default: realistic)
    :param should_remesh: Whether to remesh the model (default: True)
    :param mode: Mode of the generation (default: preview)
    :return: Response JSON containing model details
    """
    url = "https://api.meshy.ai/openapi/v2/text-to-3d"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }
    payload = {
        "mode": mode,
        "prompt": prompt,
        "art_style": art_style,
        "should_remesh": should_remesh,
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        print("3D Model generated successfully!")
        return response.json()
    except requests.exceptions.RequestException as e:
        print("Error generating 3D model:", e)
        return None

def save_model_file(model_data, output_path="model.glb"):
    """
    Save the generated 3D model to a file.

    :param model_data: Binary content of the model file
    :param output_path: Path to save the file
    """
    try:
        with open(output_path, "wb") as file:
            file.write(model_data)
        print(f"Model saved successfully to {output_path}")
    except IOError as e:
        print("Error saving the model file:", e)

if __name__ == "__main__":
    # Step 1: Input the prompt for the 3D model
    prompt = input("Enter a prompt for the 3D model: ")
    api_key = input("Enter your API key: ")

    # Step 2: Generate the 3D model
    model_response = generate_3d_model_from_prompt(prompt=prompt, api_key=api_key)

    # Step 3: Save the model to a file (if generation succeeded)
    if model_response and "model_file_url" in model_response:
        model_url = model_response["model_file_url"]
        try:
            model_file_response = requests.get(model_url)
            model_file_response.raise_for_status()
            save_model_file(model_data=model_file_response.content, output_path="model.glb")
        except requests.exceptions.RequestException as e:
            print("Error downloading the model file:", e)

Enter a prompt for the 3D model: bright red t-shirt 
Enter your API key: msy_2mwcJcgdpJeiW4bSpGUcq49By6hr5y6FaYaH
3D Model generated successfully!


In [ ]:
print(model_response)

{'result': '0193e3ec-c705-7963-999b-77af8520dfe1'}


In [ ]:
import requests
import json
import time

def generate_3d_model_from_prompt(prompt, api_key, art_style="realistic", should_remesh=True, mode="preview"):
    """
    Generate a 3D model based on a textual prompt using the Meshy API.

    :param prompt: A textual description of the 3D model to generate
    :param api_key: API key for authentication
    :param art_style: Style of the generated 3D model (default: realistic)
    :param should_remesh: Whether to remesh the model (default: True)
    :param mode: Mode of the generation (default: preview)
    :return: Model ID of the generated 3D model
    """
    url = "https://api.meshy.ai/openapi/v2/text-to-3d"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }
    payload = {
        "mode": mode,
        "prompt": prompt,
        "art_style": art_style,
        "should_remesh": should_remesh,
        "topology": "quad",
        "target_polycount": 4000
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        result = response.json()
        model_id = result.get("result")
        print("3D Model generated successfully! Model ID:", model_id)
        return model_id
    except requests.exceptions.RequestException as e:
        print("Error generating 3D model:", e)
        return None

def fetch_model_status_and_links(model_id, api_key, retry_interval=10, max_retries=30):
    """
    Poll the API to fetch the status and downloadable links for the generated 3D model.

    :param model_id: The ID of the 3D model
    :param api_key: API key for authentication
    :param retry_interval: Time (in seconds) to wait between retries
    :param max_retries: Maximum number of retries before giving up
    :return: Dictionary containing model URLs or None if the model generation fails
    """
    url = f"https://api.meshy.ai/openapi/v2/text-to-3d/{model_id}"
    headers = {
        "Authorization": f"Bearer {api_key}"
    }

    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            model_data = response.json()
            status = model_data.get("status")
            print(f"Attempt {attempt + 1}/{max_retries}: Model status - {status}")

            if status == "SUCCEEDED":
                print("Model generation completed successfully!")
                return model_data.get("model_urls", {})
            elif status == "FAILED":
                print("Model generation failed.")
                return None

            time.sleep(retry_interval)
        except requests.exceptions.RequestException as e:
            print("Error fetching model status:", e)
            return None

    print("Max retries reached. Model generation did not complete.")
    return None

def save_model_file(file_url, output_path="model.glb"):
    """
    Download and save the 3D model file.

    :param file_url: URL of the model file to download
    :param output_path: Path to save the file
    """
    try:
        response = requests.get(file_url)
        response.raise_for_status()
        with open(output_path, "wb") as file:
            file.write(response.content)
        print(f"Model saved successfully to {output_path}")
    except requests.exceptions.RequestException as e:
        print("Error downloading the model file:", e)

if __name__ == "__main__":
    # Step 1: Input the prompt and API key
    prompt = "Red Colored T-Shirt"
    api_key = "msy_2mwcJcgdpJeiW4bSpGUcq49By6hr5y6FaYaH"

    # Step 2: Generate the 3D model and get the model ID
    model_id = generate_3d_model_from_prompt(prompt=prompt, api_key=api_key)

    # Step 3: Poll for model status and download the GLB file (if generation succeeded)
    if model_id:
        model_links = fetch_model_status_and_links(model_id=model_id, api_key=api_key)
        if model_links and "glb" in model_links:
            save_model_file(file_url=model_links["glb"], output_path="model.glb")


Error generating 3D model: 400 Client Error: Bad Request for url: https://api.meshy.ai/openapi/v2/text-to-3d


#inference

## Text to 3D model Generation

In [ ]:
import requests
import json
import time

def generate_task(prompt, api_key, mode="preview", art_style="realistic", should_remesh=True, preview_task_id=None, enable_pbr=False):
    """
    Generate a 3D model task (preview or refine) using the Meshy API.

    :param prompt: A textual description of the 3D model to generate
    :param api_key: API key for authentication
    :param mode: Mode of the task ("preview" or "refine")
    :param art_style: Style of the generated 3D model (default: realistic)
    :param should_remesh: Whether to remesh the model (default: True)
    :param preview_task_id: The ID of the preview task (required for refine mode)
    :param enable_pbr: Enable PBR for refine mode (default: False)
    :return: Task ID of the generated task
    """
    url = "https://api.meshy.ai/openapi/v2/text-to-3d"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }

    payload = {
        "mode": mode,
    }

    if mode == "preview":
        payload.update({
            "prompt": prompt,
            "art_style": art_style,
            "should_remesh": should_remesh,
            "enable_pbr": enable_pbr,
            "ai_model": "meshy-4",
            "topology": "triangle",
            "target_polycount": 20000,
            # "negative_prompt": "low quality, low resolution, high poly, ugly, human",
            "texture_richness": "high"

        })
    elif mode == "refine":
        if not preview_task_id:
            raise ValueError("preview_task_id is required for refine mode.")
        payload.update({
            "preview_task_id": preview_task_id,
            "enable_pbr": enable_pbr,
            "art_style": art_style,
            "should_remesh": should_remesh,
            "topology": "triangle",
            "ai_model": "meshy-4",
            "target_polycount": 20000,
            # "negative_prompt": "low quality, low resolution, high poly, ugly, human",
            "texture_richness": "high"
        })
    else:
        raise ValueError("Invalid mode. Choose either 'preview' or 'refine'.")

    try:
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        result = response.json()
        task_id = result.get("result")
        # print(f"{mode.capitalize()} task created successfully! Task ID: {task_id}")
        return task_id
    except requests.exceptions.RequestException as e:
        print(f"Error creating {mode} task:", e)
        return None

def fetch_task_status(task_id, api_key, mode, retry_interval=10, max_retries=30):
    """
    Poll the API to fetch the status and downloadable links for the given task.

    :param task_id: The ID of the task (preview or refine)
    :param api_key: API key for authentication
    :param retry_interval: Time (in seconds) to wait between retries
    :param max_retries: Maximum number of retries before giving up
    :return: Dictionary containing model URLs or None if the task fails
    """
    url = f"https://api.meshy.ai/openapi/v2/text-to-3d/{task_id}"
    headers = {
        "Authorization": f"Bearer {api_key}"
    }

    for attempt in range(1, max_retries + 1):
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            task_data = response.json()
            status = task_data.get("status")
            if mode == "preview":
                print(f"Buidling layers: Job status - {status}")
            elif mode == "refine":
                print(f"Adding Textures: Job status - {status}")

            if status == "SUCCEEDED":
                if mode == "refine":
                    print("model build successfuly!")
                return task_data.get("model_urls", {})
            elif status == "FAILED":
                print("Task failed.")
                return None

            time.sleep(retry_interval)
        except requests.exceptions.RequestException as e:
            print("Error fetching task status:", e)
            return None

    print("Max retries reached. Task did not complete.")
    return None

def save_model_file(file_url, mode = "preview", output_path="model.glb"):
    """
    Download and save the 3D model file.

    :param file_url: URL of the model file to download
    :param output_path: Path to save the file
    """
    try:
        response = requests.get(file_url)
        response.raise_for_status()
        with open(output_path, "wb") as file:
            file.write(response.content)
        if mode == "refine":
            print(f"Model saved successfully to {output_path}")
    except requests.exceptions.RequestException as e:
        print("Error downloading the model file:", e)

def main():
    # Step 1: Input the prompt and API key
    # prompt = input("Enter a prompt for the 3D model: ")
    prompt = input("Enter prompt here:")
    api_key = input("Enter your Api Key:")

    # Step 2: Generate the Preview Task and get the preview task ID
    preview_task_id = generate_task(prompt=prompt, api_key=api_key, mode="preview", enable_pbr=True)

    # Step 3: Poll for Preview Task Status
    if preview_task_id:
        preview_links = fetch_task_status(preview_task_id, api_key, mode="preview")
        if preview_links and "glb" in preview_links:
            # print("Creating your model")
            save_model_file(file_url=preview_links["glb"], output_path="test_model.glb")
        else:
            print("some error.")
            return

        # Step 4: Generate the Refine Task using the preview task ID
        refine_task_id = generate_task(
            prompt=prompt,
            api_key=api_key,
            mode="refine",
            preview_task_id=preview_task_id,
            enable_pbr=True  # Set to True as per your refine task requirements
        )

        # Step 5: Poll for Refine Task Status
        if refine_task_id:
            refine_links = fetch_task_status(refine_task_id, api_key, mode="refine")
            if refine_links and "glb" in refine_links:
                print("model generated converting to glb...")
                save_model_file(file_url=refine_links["glb"], mode = "refine",output_path="final_model.glb")
            else:
                print("creating model failed")
    else:
        print("Failed to create a  task. Exiting.")

if __name__ == "__main__":
    main()



enter prompt hereA refine Model of Skin color hoodie
Buidling layers: Job status - IN_PROGRESS
Buidling layers: Job status - IN_PROGRESS
Buidling layers: Job status - IN_PROGRESS
Buidling layers: Job status - IN_PROGRESS
Buidling layers: Job status - SUCCEEDED
Adding Textures: Job status - IN_PROGRESS
Adding Textures: Job status - IN_PROGRESS
Adding Textures: Job status - IN_PROGRESS
Adding Textures: Job status - IN_PROGRESS
Adding Textures: Job status - IN_PROGRESS
Adding Textures: Job status - IN_PROGRESS
Adding Textures: Job status - IN_PROGRESS
Adding Textures: Job status - IN_PROGRESS
Adding Textures: Job status - IN_PROGRESS
Adding Textures: Job status - IN_PROGRESS
Adding Textures: Job status - IN_PROGRESS
Adding Textures: Job status - IN_PROGRESS
Adding Textures: Job status - IN_PROGRESS
Adding Textures: Job status - IN_PROGRESS
Adding Textures: Job status - IN_PROGRESS
Adding Textures: Job status - IN_PROGRESS
Adding Textures: Job status - IN_PROGRESS
Adding Textures: Job stat

# Image-to-3D Generation

In [ ]:
import requests
import json
import time
import base64


def generate_image_to_3d_task(image_path, api_key, enable_pbr=True, should_remesh=True, should_texture=True):
    """
    Generate a 3D model from an image using the Meshy API.

    :param image_path: Path to the image file to convert
    :param api_key: API key for authentication
    :param enable_pbr: Enable PBR for the generated model (default: True)
    :param should_remesh: Whether to remesh the model (default: True)
    :param should_texture: Whether to apply textures to the model (default: True)
    :return: Task ID of the generated task
    """
    url = "https://api.meshy.ai/openapi/v1/image-to-3d"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }

    with open(image_path, "rb") as image_file:
        image_data = base64.b64encode(image_file.read()).decode("utf-8")

    payload = {
        "image_url": f"data:image/png;base64,{image_data}",
        "enable_pbr": enable_pbr,
        "should_remesh": should_remesh,
        "should_texture": should_texture,
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        result = response.json()
        task_id = result.get("result")
        print(f"Image-to-3D task created successfully! Task ID: {task_id}")
        return task_id
    except requests.exceptions.RequestException as e:
        print("Error creating Image-to-3D task:", e)
        return None


def fetch_task_status_image_to_3d(task_id, api_key, retry_interval=10, max_retries=30):
    """
    Poll the API to fetch the status and downloadable links for the given Image-to-3D task.

    :param task_id: The ID of the Image-to-3D task
    :param api_key: API key for authentication
    :param retry_interval: Time (in seconds) to wait between retries
    :param max_retries: Maximum number of retries before giving up
    :return: Dictionary containing model URLs or None if the task fails
    """
    url = f"https://api.meshy.ai/openapi/v1/image-to-3d/{task_id}"
    headers = {
        "Authorization": f"Bearer {api_key}"
    }

    for attempt in range(1, max_retries + 1):
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            task_data = response.json()
            status = task_data.get("status")
            print(f"Task status: {status}")

            if status == "SUCCEEDED":
                print("Image-to-3D task completed successfully!")
                return task_data.get("model_urls", {})
            elif status == "FAILED":
                print("Task failed.")
                return None

            time.sleep(retry_interval)
        except requests.exceptions.RequestException as e:
            print("Error fetching task status:", e)
            return None

    print("Max retries reached. Task did not complete.")
    return None


def save_model_file(file_url, output_path):
    """
    Download and save the 3D model file.

    :param file_url: URL of the model file to download
    :param output_path: Path to save the file
    """
    try:
        response = requests.get(file_url)
        response.raise_for_status()
        with open(output_path, "wb") as file:
            file.write(response.content)
        print(f"Model saved successfully to {output_path}")
    except requests.exceptions.RequestException as e:
        print("Error downloading the model file:", e)


def main():
    # Input the image path and API key
    image_path = input("Enter the path to the image file: ")
    api_key = "msy_2mwcJcgdpJeiW4bSpGUcq49By6hr5y6FaYaH"  # Replace with your actual API key

    # Generate the Image-to-3D Task and get the task ID
    task_id = generate_image_to_3d_task(image_path=image_path, api_key=api_key)

    # Poll for Task Status
    if task_id:
        model_links = fetch_task_status_image_to_3d(task_id, api_key)
        if model_links and "glb" in model_links:
            save_model_file(file_url=model_links["glb"], output_path="image_to_3d_model.glb")
        else:
            print("Failed to generate the 3D model.")
    else:
        print("Failed to create an Image-to-3D task. Exiting.")


if __name__ == "__main__":
    main()
